In [ ]:
%pip install -U pip setuptools wheel
%pip install -U spacy spacy-transformers
!python -m spacy download en_core_web_sm

In [1]:
import rich, json
import spacy
from spacy.tokens import DocBin
import os

In [2]:
with open(f'{os.getcwd()}/labels_en2.json','r') as f:
    data = json.loads(f.read())

In [3]:
nlp = spacy.load("en_core_web_sm") 
doc_bin = DocBin()

/home/ronji/repos/ner-api/apienv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def has_overlap(span1, span2):
    return not (span1.end <= span2.start or span2.end <= span1.start)

overlap_count = 0
empty_count = 0
ent_count = 0
db = DocBin()
for d in data:
    debug_id = d["id"]
    text = d["content"]
    labels = d["labels"]
    doc = nlp(text)
    ents = []
    for l in labels:
        start, end, label, value = l["start"], l["end"], l["label"], l["value"]
        span = doc.char_span(start, end, label=label, alignment_mode='expand')
        if span is None:
            empty_count +=1
            continue
        if any(has_overlap(span, existing_span) for existing_span in ents):
            overlap_count +=1
            continue
        ents.append(span)
        ent_count += len(ents)
    doc.ents = ents 
    db.add(doc) 
print(ent_count,overlap_count, empty_count)
db.to_disk("train.spacy")

In [5]:
! python -m spacy init config config.cfg --lang en --pipeline ner --optimize accuracy

ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: accuracy
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
! python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy